In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D, Dense, Activation, Dropout, Embedding,Conv1D
import random
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences


    

In [3]:
#loading file json
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = '/kaggle/input/friends/intents.json'

intents = load_json_file(filename)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [5]:
# Create an empty DataFrame
df = pd.DataFrame(columns=['Pattern', 'Tag'])

In [6]:
df

,Pattern,Tag


In [7]:
def extract_json_info(json_file, df):
    # Iterate over each intent in the JSON file
    for intent in json_file['intents']:
        # Iterate over each pattern in the current intent
        for pattern in intent['patterns']:
            # Create a list containing the pattern and its associated tag
            sentence_tag = [pattern, intent['tag']]
            # Append the pattern and tag to the DataFrame
            df.loc[len(df.index)] = sentence_tag
     # Return the updated DataFrame            
    return df

df = extract_json_info(intents, df)
df.head(20)

,Pattern,Tag
0,Hi,greeting
1,How are you?,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
5,What's up,greeting
6,how are ya,greeting
7,heyy,greeting
8,whatsup,greeting
9,??? ??? ??,greeting


In [9]:
#create list of unique tags from the 'Tag' colum
labels = df['Tag'].unique().tolist()
labels = [s.strip() for s in labels]

In [10]:
labels

['greeting',
 'goodbye',
 'creator',
 'name',
 'hours',
 'number',
 'course',
 'fees',
 'location',
 'hostel',
 'event',
 'document',
 'floors',
 'syllabus',
 'library',
 'infrastructure',
 'canteen',
 'menu',
 'placement',
 'ithod',
 'computerhod',
 'extchod',
 'principal',
 'sem',
 'admission',
 'scholarship',
 'facilities',
 'office intake',
 'uniform',
 'committee',
 'random',
 'swear',
 'vacation',
 'sports',
 'salutaion',
 'task',
 'ragging',
 'hod',
 'leave',
 'days']

In [11]:
df.shape

(419, 2)

In [12]:
len(df)

419

In [13]:
#info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 419 entries, 0 to 418
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  419 non-null    object
 1   Tag      419 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


In [14]:
#count each tag
tag_counts = df['Tag'].value_counts()
print(tag_counts)

Tag
course            27
scholarship       26
fees              23
hostel            22
hours             17
creator           16
number            15
library           14
location          14
salutaion         13
document          13
name              13
vacation          12
goodbye           12
sem               11
event             11
canteen           11
ragging           10
greeting          10
swear              9
uniform            9
office intake      9
days               9
placement          9
principal          7
floors             7
syllabus           7
sports             7
menu               7
committee          6
admission          6
task               6
facilities         5
leave              5
ithod              4
extchod            4
computerhod        4
random             3
infrastructure     3
hod                3
Name: count, dtype: int64


In [15]:
len(labels)

40

In [16]:
# Tokenize the text patterns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Pattern'])

# Convert text patterns to numerical sequences
X = tokenizer.texts_to_sequences(df['Pattern'])

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Convert tags to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Tag'])

In [17]:
X_padded

array([[166,   0,   0, ...,   0,   0,   0],
       [ 12,   9,   5, ...,   0,   0,   0],
       [  1, 167,  25, ...,   0,   0,   0],
       ...,
       [115,  50,   0, ...,   0,   0,   0],
       [289,   0,   0, ...,   0,   0,   0],
       [ 92, 290, 291, ...,   0,   0,   0]], dtype=int32)

In [18]:
y

array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7, 12, 12, 12, 12, 12, 12, 12, 36, 36, 36, 36,
       36, 36, 36, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       18, 18, 18,  1,  1

In [20]:
print(X_padded.shape)
print(y.shape)

(419, 9)
(419,)


In [21]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [35]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [37]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0204 - loss: 43.2093 - val_accuracy: 0.0000e+00 - val_loss: 15.5541
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0294 - loss: 28.2782 - val_accuracy: 0.0000e+00 - val_loss: 9.0889
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0426 - loss: 21.5787 - val_accuracy: 0.0238 - val_loss: 6.4169
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0359 - loss: 17.8269 - val_accuracy: 0.0119 - val_loss: 5.0289
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0130 - loss: 13.2501 - val_accuracy: 0.0119 - val_loss: 4.1550
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0488 - loss: 11.3660 - val_accuracy: 0.0119 - val_loss: 3.8243
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0142 - loss: 9.7166 - val_accuracy: 0.0119 - val_loss: 3.7861
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0369 - loss: 7.8412 - val_accu

In [39]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0805 - loss: 3.5813 
Validation Accuracy: 0.0595


In [42]:
#saving model
model.save('chatbot.h5')

In [47]:
# Load the saved model
loaded_model = load_model('chatbot.h5')

In [49]:
# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to generate response
def generate_response(sentence, loaded_model, tokenizer, max_sequence_length, label_encoder):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict probabilities for all classes
    predicted_probabilities = loaded_model.predict(preprocessed_input)
    # Get the index of the class with the highest probability
    predicted_label_index = np.argmax(predicted_probabilities, axis=-1)
    # Get the predicted tag based on the index
    predicted_tag = label_encoder.inverse_transform(predicted_label_index)
    return predicted_tag


In [59]:
#Add sentence and get their predicted Tag
# Example of conversation loop
print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from the chatbot
    response = generate_response(user_input, loaded_model, tokenizer, max_sequence_length, label_encoder)
    print("Chatbot:", response[0])

Chatbot: Hello! How can I assist you today? (Type 'exit' to end)


You:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot: course


You:  how are you


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Chatbot: course


You:  exit


Chatbot: Goodbye! Have a great day!


In [57]:
#getting all patterns of the predicted Tag
# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence
# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents, df):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    # Get the response based on the predicted tag
    if predicted_tag[0] in df['Tag'].values:
        response_row = df[df['Tag'] == predicted_tag[0]].iloc[0]
        response = response_row['Tag'] + ": " + df['Pattern']
    else:
        response = intents.get(predicted_tag[0], "I'm sorry, I didn't understand that.")
    return response
# Example usage
sentence = "Hi"
response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents, df)
print("Chatbot:", response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: 0                      course: Hi
1            course: How are you?
2        course: Is anyone there?
3                   course: Hello
4                course: Good day
                  ...            
414              course: two week
415             course: Five days
416              course: two days
417             course: tommorrow
418    course: day after tomorrow
Name: Pattern, Length: 419, dtype: object


In [58]:
#Now i can chat with my chatbot 
# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence
# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    
    # Iterate through the intents to find the matching tag
    for intent in intents["intents"]:
        if intent["tag"] == predicted_tag[0]:
            # Select a random response from the list of responses
            response = random.choice(intent["responses"])
            return response
    
    # If no matching intent is found, return a default message
    return "I'm sorry, I didn't understand that."
print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    # Get user input
    sentence = input("You: ")
    # Check if the user wants to exit
    if sentence.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from chatbot
    response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents)
    print("Chatbot:", response)

Chatbot: Hello! How can I assist you today? (Type 'exit' to end)


You:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Our rganization offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.


You:  i need leave


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Our rganization offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.


You:  exit


Chatbot: Goodbye! Have a great day!


In [ ]:


def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict probabilities
    predicted_probabilities = loaded_model.predict(preprocessed_input)[0]
    # Get index with highest probability
    predicted_label_index = predicted_probabilities.argmax()
    # Convert index to tag
    predicted_tag = label_encoder.inverse_transform([predicted_label_index])[0]
    
    # Iterate through the intents to find the matching tag
    for intent in intents["intents"]:
        if intent["tag"] == predicted_tag:
            # Select a random response from the list of responses
            response = random.choice(intent["responses"])
            return response
    
    # If no matching intent is found, return a default message
    return "I'm sorry, I didn't understand that."

print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    # Get user input
    sentence = input("You: ")
    # Check if the user wants to exit
    if sentence.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from chatbot
    response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents)
    print("Chatbot:", response)
